<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/all_town_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 町の全スコア
- input:
    - population_master.csv
        - 町丁ごとの位置
    - train_station.csv
        - 最寄り駅
        - 最寄り駅から主要駅までの時間
    - childcare_score.csv
        - 子育てスコア
            - 保育園スコア
            - 幼稚園スコア
            - 児童館スコア
    - education_score.csv
        - 教育スコア
            - 小学校スコア
            - 中学校スコア
            - 図書館スコア
    - convenience_score.csv
        - 利便性スコア
            - 買い物スコア
            - 駅スコア
    - environment_score.csv
        - 周辺環境スコア
            - 公園スコア
            - 運動場スコア
            - 体育館スコア
            - プールスコア
    - medical_score.csv
        - 医療スコア
            - 病院スコア
            - 小児科スコア
            - 薬局スコア
    - security_score.csv
        - 治安防災スコア
    - population_core.csv
        - ○○割合スコア
    - land_price_score.csv
        - 家賃
    - town_feature.csv
        - 町の特徴量
- output:
    - all_town_score.csv
        - 町丁ごとの全スコアデータ

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [57]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [58]:
output_file = 'all_town_score.csv'

### 人口マスタ

In [59]:
df_population = pd.read_csv(os.path.join(DIR_NAME, 'population_master.csv'))[[
    'town_id', '市区町村名', '町名', '町丁目', '対象'
]].set_index('town_id')
print(df_population.shape)
df_population.head()

(3145, 4)


,市区町村名,町名,町丁目,対象
town_id,,,,
101003,千代田区,丸の内,丸の内１丁目,0
101004,千代田区,丸の内,丸の内２丁目,0
101005,千代田区,丸の内,丸の内３丁目,0
101007,千代田区,大手町,大手町１丁目,0
101008,千代田区,大手町,大手町２丁目,0


### 地理マスタ

In [60]:
df_geo_master = pd.read_csv(os.path.join(DIR_NAME, 'geocoding.csv'))[[
    'town_id', '緯度', '経度'
]].set_index('town_id')
print(df_geo_master.shape)
df_geo_master.head()

(3145, 2)


,緯度,経度
town_id,,
101003,35.681560,139.767201
101004,35.680022,139.763447
101005,35.676952,139.763476
101007,35.688069,139.763929
101008,35.686301,139.768087


### 最寄り駅

In [61]:
major_station = ['東京駅', '上野駅', '池袋駅', '新宿駅', '渋谷駅', '品川駅']
df_train_station = pd.read_csv(os.path.join(DIR_NAME, 'train_station_route.csv'))[
    ['no', 'name'] + major_station
].rename(columns={'no':'town_id', 'name':'最寄り駅'}).set_index('town_id')
print(df_train_station.shape)

(3141, 7)


In [62]:
# 最寄り駅と目的地が同じ場合に0分になるように補正
for station in major_station:
    offset = df_train_station[df_train_station['最寄り駅'] == station].iloc[0][station]
    print(station, offset)
    df_train_station[station] = df_train_station[station] - offset

東京駅 4.0
上野駅 4.0
池袋駅 2.0
新宿駅 2.0
渋谷駅 1.0
品川駅 2.0


In [63]:
df_train_station.head()

,最寄り駅,東京駅,上野駅,池袋駅,新宿駅,渋谷駅,品川駅
town_id,,,,,,,
101003,東京駅,0.0,9.0,19.0,18.0,24.0,13.0
101004,東京駅,0.0,9.0,19.0,18.0,24.0,13.0
101005,有楽町駅,4.0,12.0,24.0,25.0,24.0,13.0
101007,大手町駅,4.0,17.0,18.0,23.0,22.0,21.0
101008,大手町駅,4.0,17.0,18.0,23.0,22.0,21.0


In [64]:
from scipy.stats import norm

def rank_score(sr_data):
    rank = sr_data.rank()
    count = rank.max() 
    x = (rank + 1)/(count + 1)
    max_val = 5
    y = norm.ppf(x, loc=0.5, scale=0.14)*max_val
    return np.clip(y, 0, max_val)

### 子育てスコア

In [65]:
df_childcare = pd.read_csv(os.path.join(DIR_NAME, 'childcare_score.csv'))[[
    'town_id', '保育園スコア', '児童館スコア', '子育てスコア'
]].set_index('town_id')
print(df_childcare.shape)

for col in df_childcare.columns[:-1]:
    print(col)
    df_childcare[col] = rank_score(df_childcare[col]).round(2)
df_childcare.head()

(3145, 3)
保育園スコア
児童館スコア


,保育園スコア,児童館スコア,子育てスコア
town_id,,,
101003,1.04,1.23,0.20
101004,1.01,0.89,0.10
101005,1.07,0.89,0.10
101007,1.31,1.07,0.07
101008,1.43,1.50,0.22


### 教育スコア

In [66]:
df_education = pd.read_csv(os.path.join(DIR_NAME, 'education_score.csv')).set_index('town_id')
print(df_education.shape)

for col in df_education.columns[:-1]:
    print(col)
    df_education[col] = rank_score(df_education[col]).round(2)
df_education.head()

(3145, 7)
小学校スコア
中学校スコア
幼稚園スコア
学習塾スコア
図書館スコア
書店スコア


,小学校スコア,中学校スコア,幼稚園スコア,学習塾スコア,図書館スコア,書店スコア,教育スコア
town_id,,,,,,,
101003,3.09,2.36,2.54,2.28,1.33,3.64,2.93
101004,2.80,2.36,2.28,1.99,1.48,3.04,2.06
101005,2.10,2.37,2.28,2.13,1.88,3.54,1.54
101007,2.87,2.44,2.08,1.59,2.27,3.87,2.58
101008,3.33,2.37,2.39,1.73,2.15,3.30,3.06


### 利便性スコア

In [67]:
df_convenience = pd.read_csv(os.path.join(DIR_NAME, 'convenience_score.csv'))[[
    'town_id', '買い物スコア', '駅スコア', '飲食店スコア', '利便性スコア', '最寄駅(距離)',
    '銭湯(数)', 'カラオケ(数)', 'ネットカフェ(数)', 'クリーニング(数)', '本DVD(数)',
    '100円ショップ(数)', 'カフェ(数)', 'カフェ(チェーン店)(数)', 'ファストフード(数)', 'レストラン(数)',
    'レストラン(チェーン店)(数)', '飲み屋(数)'
]].set_index('town_id')

for col in ['買い物スコア', '駅スコア', '飲食店スコア']:
    df_convenience[col] = rank_score(df_convenience[col]).round(2)

df_convenience['レストラン(数)'] = df_convenience['レストラン(数)'] + df_convenience['レストラン(チェーン店)(数)']
df_convenience['カフェ(数)'] = df_convenience['カフェ(数)'] + df_convenience['カフェ(チェーン店)(数)']
df_convenience['チェーン店(数)'] = df_convenience['レストラン(チェーン店)(数)'] + df_convenience['カフェ(チェーン店)(数)'] + df_convenience['ファストフード(数)']
df_convenience = df_convenience.drop(columns=['レストラン(チェーン店)(数)', 'カフェ(チェーン店)(数)'])
print(df_convenience.shape)
df_convenience.head()

(3145, 16)


,買い物スコア,駅スコア,飲食店スコア,利便性スコア,最寄駅(距離),銭湯(数),カラオケ(数),ネットカフェ(数),クリーニング(数),本DVD(数),100円ショップ(数),カフェ(数),ファストフード(数),レストラン(数),飲み屋(数),チェーン店(数)
town_id,,,,,,,,,,,,,,,,
101003,2.90,5.00,3.39,4.77,0.037,0,2,0,0,3,1,12,1,22,31,5
101004,2.43,3.89,3.68,4.42,0.359,0,0,0,0,3,1,16,2,23,36,7
101005,2.43,4.01,3.80,4.46,0.219,1,2,0,0,2,1,19,2,24,39,7
101007,2.43,3.82,3.25,4.18,0.308,1,0,0,1,3,0,16,1,18,30,9
101008,2.89,3.55,3.19,4.57,0.447,0,0,0,2,3,0,14,1,18,25,8


### 周辺環境スコア

In [68]:
df_environment = pd.read_csv(os.path.join(DIR_NAME, 'environment_score.csv')).set_index('town_id')
print(df_environment.shape)

for col in df_environment.columns[:-1]:
    print(col)
    df_environment[col] = rank_score(df_environment[col]).round(2)
df_environment.head()

(3145, 6)
公園スコア
運動場スコア
体育館スコア
プールスコア
フィットネススコア


,公園スコア,運動場スコア,体育館スコア,プールスコア,フィットネススコア,周辺環境スコア
town_id,,,,,,
101003,1.92,1.62,3.16,3.16,3.58,1.85
101004,1.69,1.37,2.92,2.94,3.58,1.31
101005,1.69,1.18,2.44,2.71,4.04,0.95
101007,1.93,1.58,4.05,4.01,4.04,2.33
101008,2.00,1.89,4.20,4.14,3.58,2.53


### 医療スコア

In [69]:
df_medical = pd.read_csv(os.path.join(DIR_NAME, 'medical_score.csv'))[[
    'town_id', '病院スコア', '小児科スコア', '薬局スコア', '医療スコア'
]].set_index('town_id')
print(df_medical.shape)

for col in df_medical.columns[:-1]:
    print(col)
    df_medical[col] = rank_score(df_medical[col]).round(2)
df_medical.head()

(3145, 4)
病院スコア
小児科スコア
薬局スコア


,病院スコア,小児科スコア,薬局スコア,医療スコア
town_id,,,,
101003,2.96,3.48,2.95,4.21
101004,2.05,2.63,2.02,1.66
101005,3.16,3.04,2.89,4.03
101007,3.44,2.68,3.47,4.26
101008,3.27,2.75,3.20,4.08


### 治安防災

In [70]:
df_security = pd.read_csv(os.path.join(DIR_NAME, 'security_score.csv'))[[
    'town_id', '犯罪スコア', '警察スコア', '交通事故スコア', '防災スコア', '治安防災スコア'
]].set_index('town_id')
df_security['治安スコア'] = df_security['治安防災スコア']   # 互換性のため
print(df_security.shape)

for col in df_security.columns[:-2]:
    print(col)
    df_security[col] = rank_score(df_security[col]).round(2)
df_security.head()

(3145, 6)
犯罪スコア
警察スコア
交通事故スコア
防災スコア


,犯罪スコア,警察スコア,交通事故スコア,防災スコア,治安防災スコア,治安スコア
town_id,,,,,,
101003,1.26,3.43,2.23,5.0,1.53,1.53
101004,2.18,4.59,2.54,5.0,3.61,3.61
101005,1.96,4.59,1.91,5.0,2.09,2.09
101007,1.01,2.37,1.37,5.0,0.09,0.09
101008,2.05,1.65,1.79,5.0,0.29,0.29


### 人口分布

In [71]:
df_pop_rate = pd.read_csv(os.path.join(DIR_NAME, 'population_score.csv'))[[
    'town_id', '乳幼児割合スコア', '小学生割合スコア', '中学生割合スコア', '高校生割合スコア', 
    '10代割合スコア', '20代割合スコア', '30代割合スコア', '40代割合スコア', '50代割合スコア', '60代割合スコア', 
    '10代_子割合スコア', '20代_子割合スコア', '30代_子割合スコア', '40代_子割合スコア', '50代_子割合スコア',
]].set_index('town_id')
print(df_pop_rate.shape)

for col in df_pop_rate.columns:
    df_pop_rate[col] = rank_score(df_pop_rate[col]).round(2)
df_pop_rate.head()

(3145, 15)


,乳幼児割合スコア,小学生割合スコア,中学生割合スコア,高校生割合スコア,10代割合スコア,20代割合スコア,30代割合スコア,40代割合スコア,50代割合スコア,60代割合スコア,10代_子割合スコア,20代_子割合スコア,30代_子割合スコア,40代_子割合スコア,50代_子割合スコア
town_id,,,,,,,,,,,,,,,
101003,1.1,1.1,1.12,1.1,1.1,1.07,1.07,1.07,1.07,1.08,1.08,1.07,1.07,1.07,1.07
101004,1.1,1.1,1.12,1.1,1.1,1.07,1.07,1.07,1.07,1.08,1.08,1.07,1.07,1.07,1.07
101005,1.1,1.1,1.12,1.1,1.1,1.07,1.07,1.07,1.07,1.08,1.08,1.07,1.07,1.07,1.07
101007,1.1,1.1,1.12,1.1,1.1,1.07,1.07,1.07,1.07,1.08,1.08,1.07,1.07,1.07,1.07
101008,1.1,1.1,1.12,1.1,1.1,1.07,1.07,1.07,1.07,1.08,1.08,1.07,1.07,1.07,1.07


### 家賃

In [72]:
df_rent_price = pd.read_csv(os.path.join(DIR_NAME, 'land_price_score.csv'))\
    .rename(columns={'地域ID': 'town_id'}).drop(columns=['地価', '地価スコア']).set_index('town_id')
print(df_rent_price.shape)
df_rent_price.head()

(3145, 4)


,家賃予測_ワンルーム,家賃予測_1K,家賃予測_1LDK,家賃予測_2LDK
town_id,,,,
101003,13.31,13.74,29.27,45.23
101004,13.56,13.98,29.98,46.38
101005,13.01,13.45,28.43,43.89
101007,12.52,12.99,27.09,41.73
101008,13.13,13.57,28.79,44.46


### 町の特徴量

In [73]:
df_town_feature = pd.read_csv(os.path.join(DIR_NAME, 'town_feature.csv')).set_index('town_id')
print(df_town_feature.shape)

for col in df_town_feature.columns:
    df_town_feature[col] = rank_score(df_town_feature[col]).round(2)
df_town_feature.head()

(3145, 24)


,閑静スコア,セレブスコア,繁華街スコア,団地スコア,戸建スコア,低層マンションスコア,高層マンションスコア,アパートスコア,おしゃれスコア,歴史文化スコア,下町スコア,都会スコア,郊外スコア,川スコア,自然スコア,駅近スコア,商店街スコア,散歩道スコア,学生スコア,単身スコア,カップルスコア,子育てファミリースコア,賃貸スコア,購入スコア
town_id,,,,,,,,,,,,,,,,,,,,,,,,
101003,2.16,1.03,1.64,1.82,2.21,1.69,2.44,2.54,1.63,1.63,2.03,1.63,1.88,1.92,2.07,1.92,1.63,1.63,1.78,1.81,1.63,1.75,1.67,2.17
101004,2.16,1.03,1.64,1.82,2.21,1.69,2.44,2.54,1.63,1.63,2.03,1.63,1.88,1.92,2.07,1.92,1.63,1.63,1.78,1.81,1.63,1.75,1.67,2.17
101005,2.16,1.03,1.64,1.82,2.21,1.69,2.44,2.54,1.63,1.63,2.03,1.63,1.88,1.92,2.07,1.92,1.63,1.63,1.78,1.81,1.63,1.75,1.67,2.17
101007,2.16,1.03,1.64,1.82,2.21,1.69,2.43,2.54,1.63,1.63,2.03,1.63,1.88,1.92,2.07,1.92,1.63,1.63,1.78,1.81,1.63,1.75,1.67,2.17
101008,2.16,1.03,1.64,1.82,2.21,1.69,2.44,2.54,1.63,1.63,2.03,1.63,1.88,1.92,2.07,1.92,1.63,1.63,1.78,1.81,1.63,1.75,1.67,2.17


### 各種教室

In [74]:
df_lesson_score = pd.read_csv(os.path.join(DIR_NAME, 'lesson_score.csv')).set_index('town_id')
print(df_lesson_score.shape)
df_lesson_score.head()

(3145, 4)


,音楽教室(数),英語教室(数),プログラミング(数),囲碁将棋(数)
town_id,,,,
101003,0,0,0,0
101004,0,0,0,0
101005,0,0,0,0
101007,0,2,0,0
101008,0,2,0,0


In [75]:
df_fitness_score = pd.read_csv(os.path.join(DIR_NAME, 'fitness_score.csv')).set_index('town_id')
print(df_fitness_score.shape)
df_fitness_score.head()

(3145, 5)


,ダンス教室(数),フィットネス(数),テニス教室(数),ゴルフ練習場(数),スポーツ教室(数)
town_id,,,,,
101003,0,3,0,1,1
101004,0,3,0,1,1
101005,1,5,0,2,3
101007,0,5,0,2,2
101008,0,3,0,1,0


### 全て結合

In [76]:
df_all_town_score = pd.concat([
    df_population,
    df_geo_master,
    df_train_station,
    df_childcare,
    df_education,
    df_convenience,
    df_environment,
    df_medical,
    df_security,
    df_pop_rate,
    df_rent_price,
    df_town_feature,
    df_lesson_score,
    df_fitness_score
], axis=1)
df_all_town_score.index.name = 'town_id'
print(df_all_town_score.shape)

(3145, 107)


In [77]:
num_cols = [col for col in df_all_town_score.columns if col.endswith('(数)')]
print(len(num_cols))

for col in num_cols:
    score_name = f"{col.replace('(数)', '')}スコア"
    df_all_town_score[score_name] = 0
    df_all_town_score.loc[df_all_town_score[col] > 0, score_name] = 5

df_all_town_score = df_all_town_score.drop(columns=num_cols)

20


In [78]:
score_cols = [col for col in df_all_town_score.columns if col.endswith('スコア')]
print(len(score_cols))

88


In [79]:
# 自宅から主要駅までの時間を計算(徒歩1分＝80m)
for station in major_station:
    df_all_town_score[station] = (df_all_town_score[station] + df_all_town_score['最寄駅(距離)'] * 1000 / 80 + 0.5).round()
df_all_town_score = df_all_town_score.drop(columns=['最寄駅(距離)'])

In [80]:
# データがないところは、0で埋める
for col in score_cols:
    df_all_town_score[col] = df_all_town_score[col].fillna(0)

In [81]:
list(df_all_town_score[score_cols].columns)

['保育園スコア',
 '児童館スコア',
 '子育てスコア',
 '小学校スコア',
 '中学校スコア',
 '幼稚園スコア',
 '学習塾スコア',
 '図書館スコア',
 '書店スコア',
 '教育スコア',
 '買い物スコア',
 '駅スコア',
 '飲食店スコア',
 '利便性スコア',
 '公園スコア',
 '運動場スコア',
 '体育館スコア',
 'プールスコア',
 'フィットネススコア',
 '周辺環境スコア',
 '病院スコア',
 '小児科スコア',
 '薬局スコア',
 '医療スコア',
 '犯罪スコア',
 '警察スコア',
 '交通事故スコア',
 '防災スコア',
 '治安防災スコア',
 '治安スコア',
 '乳幼児割合スコア',
 '小学生割合スコア',
 '中学生割合スコア',
 '高校生割合スコア',
 '10代割合スコア',
 '20代割合スコア',
 '30代割合スコア',
 '40代割合スコア',
 '50代割合スコア',
 '60代割合スコア',
 '10代_子割合スコア',
 '20代_子割合スコア',
 '30代_子割合スコア',
 '40代_子割合スコア',
 '50代_子割合スコア',
 '閑静スコア',
 'セレブスコア',
 '繁華街スコア',
 '団地スコア',
 '戸建スコア',
 '低層マンションスコア',
 '高層マンションスコア',
 'アパートスコア',
 'おしゃれスコア',
 '歴史文化スコア',
 '下町スコア',
 '都会スコア',
 '郊外スコア',
 '川スコア',
 '自然スコア',
 '駅近スコア',
 '商店街スコア',
 '散歩道スコア',
 '学生スコア',
 '単身スコア',
 'カップルスコア',
 '子育てファミリースコア',
 '賃貸スコア',
 '購入スコア',
 '銭湯スコア',
 'カラオケスコア',
 'ネットカフェスコア',
 'クリーニングスコア',
 '本DVDスコア',
 '100円ショップスコア',
 'カフェスコア',
 'ファストフードスコア',
 'レストランスコア',
 '飲み屋スコア',
 'チェーン店スコア',
 '音楽教室スコア',
 '英語教室スコア',
 'プログラミングスコア',
 '囲碁将棋スコア',
 'ダンス教室スコア',

### CSV出力

In [82]:
df_all_town_score.to_csv(os.path.join(DIR_NAME, output_file), index=True, encoding='utf-8_sig')

### pickle出力

In [83]:
pickle_filename = os.path.splitext(os.path.basename(output_file))[0] + '.pickle'
_tmp_all_town = df_all_town_score.reset_index()
_tmp_all_town['town_id'] = _tmp_all_town['town_id'].astype(str)
_tmp_all_town.to_pickle(os.path.join(DIR_NAME, pickle_filename))

### 読み込みテスト

In [84]:
%%time
df_tmp_csv = pd.read_csv(os.path.join(DIR_NAME, output_file))
print(df_tmp_csv.shape)

(3145, 106)
CPU times: user 44.3 ms, sys: 5.88 ms, total: 50.2 ms
Wall time: 54.8 ms


In [85]:
%%time
df_tmp_pickle = pd.read_pickle(os.path.join(DIR_NAME, pickle_filename))
print(df_tmp_pickle.shape)

(3145, 106)
CPU times: user 3.97 ms, sys: 1.94 ms, total: 5.91 ms
Wall time: 12.2 ms


In [86]:
df_tmp_csv.head(1)

,town_id,市区町村名,町名,町丁目,対象,緯度,経度,最寄り駅,東京駅,上野駅,池袋駅,新宿駅,渋谷駅,品川駅,保育園スコア,児童館スコア,子育てスコア,小学校スコア,中学校スコア,幼稚園スコア,学習塾スコア,図書館スコア,書店スコア,教育スコア,買い物スコア,...,学生スコア,単身スコア,カップルスコア,子育てファミリースコア,賃貸スコア,購入スコア,銭湯スコア,カラオケスコア,ネットカフェスコア,クリーニングスコア,本DVDスコア,100円ショップスコア,カフェスコア,ファストフードスコア,レストランスコア,飲み屋スコア,チェーン店スコア,音楽教室スコア,英語教室スコア,プログラミングスコア,囲碁将棋スコア,ダンス教室スコア,テニス教室スコア,ゴルフ練習場スコア,スポーツ教室スコア
0,101003,千代田区,丸の内,丸の内１丁目,0,35.68156,139.767201,東京駅,1.0,10.0,20.0,19.0,25.0,14.0,1.04,1.23,0.2,3.09,2.36,2.54,2.28,1.33,3.64,2.93,2.9,...,1.78,1.81,1.63,1.75,1.67,2.17,0,5,0,0,5,5,5,5,5,5,5,0,0,0,0,0,0,5,5


In [87]:
df_tmp_pickle.head(1)

,town_id,市区町村名,町名,町丁目,対象,緯度,経度,最寄り駅,東京駅,上野駅,池袋駅,新宿駅,渋谷駅,品川駅,保育園スコア,児童館スコア,子育てスコア,小学校スコア,中学校スコア,幼稚園スコア,学習塾スコア,図書館スコア,書店スコア,教育スコア,買い物スコア,...,学生スコア,単身スコア,カップルスコア,子育てファミリースコア,賃貸スコア,購入スコア,銭湯スコア,カラオケスコア,ネットカフェスコア,クリーニングスコア,本DVDスコア,100円ショップスコア,カフェスコア,ファストフードスコア,レストランスコア,飲み屋スコア,チェーン店スコア,音楽教室スコア,英語教室スコア,プログラミングスコア,囲碁将棋スコア,ダンス教室スコア,テニス教室スコア,ゴルフ練習場スコア,スポーツ教室スコア
0,101003,千代田区,丸の内,丸の内１丁目,0,35.68156,139.767201,東京駅,1.0,10.0,20.0,19.0,25.0,14.0,1.04,1.23,0.2,3.09,2.36,2.54,2.28,1.33,3.64,2.93,2.9,...,1.78,1.81,1.63,1.75,1.67,2.17,0,5,0,0,5,5,5,5,5,5,5,0,0,0,0,0,0,5,5
